In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p1/sample_submission.csv
/kaggle/input/2021-ai-w6-p1/pca_test.csv
/kaggle/input/2021-ai-w6-p1/pca_train.csv
/kaggle/input/2021-ai-w6-p1/train_label.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
device='cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

In [4]:
xtrain_data=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_train.csv")
ytrain_data=pd.read_csv("/kaggle/input/2021-ai-w6-p1/train_label.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w6-p1/pca_test.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w6-p1/sample_submission.csv")
print(xtrain_data.shape)
print(ytrain_data.shape)
print(test_data.shape)
print(submit_data.shape)

(27671, 256)
(27671, 2)
(13629, 256)
(13629, 2)


In [5]:
print(xtrain_data.info())
print(ytrain_data.max())
print(test_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27671 entries, 0 to 27670
Columns: 256 entries, 0 to 255
dtypes: float64(256)
memory usage: 54.0 MB
None
id       27670
label      197
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13629 entries, 0 to 13628
Columns: 256 entries, 0 to 255
dtypes: float64(256)
memory usage: 26.6 MB
None


In [6]:
xtrain=torch.FloatTensor(xtrain_data.values).to(device)
ytrain=torch.LongTensor(ytrain_data['label'].values).to(device)
print(xtrain.shape)
print(ytrain.shape)

torch.Size([27671, 256])
torch.Size([27671])


In [7]:
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
dataset = TensorDataset(xtrain, ytrain)
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [8]:
linear=torch.nn.Linear(256,198,bias=True).cuda()
torch.nn.init.normal_(linear.weight).cuda()
model=torch.nn.Sequential(linear).cuda()

In [9]:
loss=torch.nn.CrossEntropyLoss().cuda()
optimizer=torch.optim.Adam(model.parameters(),lr=1e-3)

epochs = 30

total_batch=len(dataloader)

for epoch in range(epochs):
    avg_cost=0
    for X, Y in dataloader:
        optimizer.zero_grad()
        hypothesis=model(X)
        cost=loss(hypothesis, Y)
        cost.backward()
        optimizer.step()
        avg_cost+=cost/total_batch
    print('epoch=', '%04d' % (epoch+1), 'cost={:.9f}'.format(avg_cost))

epoch= 0001 cost=47.590469360
epoch= 0002 cost=21.325571060
epoch= 0003 cost=14.048924446
epoch= 0004 cost=10.431659698
epoch= 0005 cost=8.187266350
epoch= 0006 cost=6.579030037
epoch= 0007 cost=5.362247944
epoch= 0008 cost=4.405576706
epoch= 0009 cost=3.652802467
epoch= 0010 cost=3.066784620
epoch= 0011 cost=2.607341290
epoch= 0012 cost=2.255014181
epoch= 0013 cost=1.985191464
epoch= 0014 cost=1.770505190
epoch= 0015 cost=1.603574395
epoch= 0016 cost=1.472288251
epoch= 0017 cost=1.362272501
epoch= 0018 cost=1.278547764
epoch= 0019 cost=1.204712868
epoch= 0020 cost=1.149457216
epoch= 0021 cost=1.098852396
epoch= 0022 cost=1.058759689
epoch= 0023 cost=1.025639415
epoch= 0024 cost=0.994404912
epoch= 0025 cost=0.971267343
epoch= 0026 cost=0.947193742
epoch= 0027 cost=0.929535985
epoch= 0028 cost=0.910519123
epoch= 0029 cost=0.895743310
epoch= 0030 cost=0.883549631


In [10]:
with torch.no_grad():
    prediction=model(xtrain)
    correct_prediction=torch.argmax(prediction,1)==ytrain
    print(correct_prediction)
    accuracy = correct_prediction.sum().item() / len(correct_prediction)
    print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))

tensor([False,  True,  True,  ...,  True,  True,  True], device='cuda:0')
The model has an accuracy of 82.53% for the training set.


In [11]:
tdata=torch.FloatTensor(test_data.values).cuda()
print(tdata.shape)

torch.Size([13629, 256])


In [12]:
prediction=torch.argmax(model(tdata),1)
print(prediction)

tensor([113,  91,  68,  ...,  23,  68, 117], device='cuda:0')


In [13]:
submit_data['label']=prediction.cpu().numpy()
print(submit_data.head())

   id  label
0   0    113
1   1     91
2   2     68
3   3     28
4   4    121


In [14]:
submit_data.to_csv("submission.csv",index=False,mode='w')